## IMPORTS 

In [ ]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torchtext.data import Example
from sklearn.metrics import f1_score
import torchtext
import os 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from unidecode import unidecode

from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

from fastai.text import * 

### Basic Parameters

In [ ]:
n_epochs = 5 # how many times to iterate over all samples
n_splits = 5 # Number of K-fold Splits
SEED = 10
debug = 0

In [ ]:
def load_and_prec():
    if debug:
        train_df = pd.read_csv("../input/train.csv")[:200]
        test_df = pd.read_csv("../input/test.csv")[:200]
    else:
        train_df = pd.read_csv("../input/train.csv")
        test_df = pd.read_csv("../input/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)

    return train_df,test_df

In [ ]:
start = time.time()
train_df,test_df = load_and_prec() 
print(time.time()-start)

# Load data for use in FastAI

## 1. Load Language model data, and Finetune Language model

In [ ]:
# Language model data : We use test_df as validation for language model
data_lm = TextLMDataBunch.from_df(path = "",train_df= train_df ,valid_df = test_df)

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [ ]:
# check how the language model performs
learn.predict("What should", n_words=10)

In [ ]:
# save the language model
learn.save_encoder('ft_enc')


## 2. Load Classifier data, and learn classifier weights

In [ ]:
def ulmfit_model_run_cv(train_df,test_df,target):
    # matrix for the out-of-fold predictions
    train_preds = np.zeros((len(train_df)))
    # matrix for the predictions on the test set
    test_preds = np.zeros((len(test_df)))
    splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED).split(train_df,target))
    for i, (train_idx, valid_idx) in enumerate(splits):
        print("Running Fold:"+str(i+1))
        train = train_df.iloc[train_idx]
        valid = train_df.iloc[valid_idx]
        # Creating Classification Data
        print("Creating Classification Data")
        data_clas = TextClasDataBunch.from_df(path ="", train_df=train, valid_df =valid,  test_df=test_df, vocab=data_lm.train_ds.vocab, bs=32,label_cols = 'target')
        
        print("Creating Classifier Object")
        learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
        learn.load_encoder('ft_enc')
        print("Fitting Classifier Object")
        learn.fit_one_cycle(1, 1e-2)
        print("Fitting Classifier Object after freezing all but last 2 layers")
        learn.freeze_to(-2)
        learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))
        print("Fitting Classifier Object - discriminative learning")
        learn.unfreeze()
        learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))
        
        valid_preds_fold = np.array(learn.get_preds(DatasetType.Valid, ordered=True)[0])[:,1]
        test_preds_fold = np.array(learn.get_preds(DatasetType.Test, ordered=True)[0])[:,1]
        train_preds[valid_idx] = valid_preds_fold
        test_preds += test_preds_fold / len(splits)
        
    return train_preds, test_preds

In [ ]:
train_oof_preds, test_preds = ulmfit_model_run_cv(train_df,test_df,train_df.target)

In [ ]:
def bestThresshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta , tmp[2]

delta, _ = bestThresshold(train_df.target,train_oof_preds)